In [236]:
import pandas as pd
import numpy as np

In [237]:
pd.set_option("display.max_columns", None)

In [238]:
# df = pd.read_excel(r"C:\Users\jober\OneDrive\Desktop\BD_CO\Base de Intermediacion Laboral 2025.xlsx",sheet_name="BASE GENERAL")
df = pd.read_excel(r"C:\Users\jober\Downloads\Base de Intermediacion Laboral 2025.xlsx",sheet_name="BASE GENERAL")


In [239]:
# Typing column the names 
df.columns = df.columns.str.lower()
df.columns = df.columns.str.replace(" ","_")

In [240]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88656 entries, 0 to 88655
Data columns (total 35 columns):
 #   Column                                                 Non-Null Count  Dtype         
---  ------                                                 --------------  -----         
 0   item                                                   5932 non-null   float64       
 1   mes_registro                                           1072 non-null   object        
 2   convenio                                               1072 non-null   object        
 3   gestión_vacante_                                       1072 non-null   object        
 4   población                                              1072 non-null   object        
 5   no.                                                    1072 non-null   float64       
 6   código_proceso                                         1072 non-null   object        
 7   nombre_de_la_empresa                                   1072 non-nul

In [241]:
df.columns

Index(['item', 'mes_registro', 'convenio', 'gestión_vacante_', 'población',
       'no.', 'código_proceso', 'nombre_de_la_empresa', 'nombre_de_la_vacante',
       'vacantes_solicitadas/puestos_de_trabajo_en_plataforma',
       'fecha_de_publicacion_de_la_vacante',
       'fecha_de_vencimiento_de_la_vacante', 'hojas_de_vida_remitidas',
       'fecha_envío_hojas_de_vida_a_la_empresa', 'nombre_del_candidato',
       'cedula', 'telefono', 'edad', 'rango_de_edad', 'nivel_educativo',
       'formacion', 'sexo', 'barrio', 'condicion', 'gestor', 'intermediador_',
       'llamada_/_convocatoria/_base_de_datos', 'observación_intermediación',
       'observación_gestión', 'observación_del_candidato',
       'mes_de_colocación_en_plataforma', 'colocado',
       'semana_publicacion_vacante', 'mes_publicacion_vacante',
       'tipo_de_vacante'],
      dtype='object')

## 0. Selecting the useful rows

In [242]:
# Cleaning NaN data. Data as 'nan' are not actual "NaN":
for column in df.columns:
    df[column] = df[column].replace('nan', pd.NA)

In [243]:
# Selecting the useful rows only
filter_mt = (df['mes_registro'].isna() & df['gestión_vacante_'].isna() & 
             df['población'].isna() & df['código_proceso'].isna())
df = df[~filter_mt]

## 1. Cleaning columns

In [244]:
# Cleaning columns
subset_columns = ['mes_registro', 'convenio', 'gestión_vacante_', 'población',
       'código_proceso', 'nombre_de_la_empresa', 'nombre_de_la_vacante',
       'nombre_del_candidato', 'nivel_educativo', 'formacion', 'sexo', 'barrio', 'condicion', 
       'gestor', 'intermediador_','llamada_/_convocatoria/_base_de_datos', 'observación_intermediación',
       'observación_gestión', 'observación_del_candidato', 'mes_de_colocación_en_plataforma', 'colocado',
       'semana_publicacion_vacante', 'mes_publicacion_vacante', 'tipo_de_vacante']

for column in subset_columns:
    df[column] = [str(i).lower() for i in df[column]]
    df[column] = [str(i).strip() for i in df[column]]

In [245]:
df = df.rename(columns={'vacantes_solicitadas/puestos_de_trabajo_en_plataforma':'puestos_de_trabajo_en_plataforma'})

In [246]:
# Cleaning column 'convenio': 
df['convenio'] = df['convenio'].replace(['ninguno', 'ninguna'],np.nan)

In [247]:
# Cleaning column 'nivel_educativo': 
dictionary = {
    'técnico':'tecnico',
    'tegnologo':'tecnologo',
    'tecnologa':'tecnologo',
    'universitaria':'universitario',
    'pfofesional':'universitario',
    'profesional':'universitario',
    'primaria':'basica primaria',
    'bachiller':'educación media',
}
df['nivel_educativo'] = df['nivel_educativo'].replace(dictionary)

In [248]:
# Cleaning column 'edad': 
df['edad'] = df['edad'].fillna(0)
df['edad'] = df['edad'].apply(lambda x:int(x))

def range_age(age):
    if  0 < age < 18:
        return '< 18'
    elif 18 <= age < 29:
        return '18-28'
    elif 29 <= age < 40:
        return '29-39'
    elif 40 <= age < 50:
        return '40-49'
    elif 50 <= age < 60:
        return '50-59'
    elif 60 <= age < 2000:
        return '> 60'
    else:
        return np.nan

df['rango_de_edad'] = df['edad'].apply(range_age)

In [249]:
# Cleaning column 'condicion': 
df['condicion'] = df['condicion'].replace(['ninguna','ninguno','nimguno'],np.nan)

In [250]:
# Cleaning column 'semana_publicacion_vacante': 
df['semana_publicacion_vacante'] = df['fecha_de_publicacion_de_la_vacante'].dt.isocalendar().week

# Cleaning column 'mes_publicacion_vacante': 
df['mes_publicacion_vacante'] = df['fecha_de_publicacion_de_la_vacante'].dt.month.fillna(0).astype(int)

# Cleaning column 'año_publicacion_vacante': 
df['año_publicacion_vacante'] = df['fecha_de_publicacion_de_la_vacante'].dt.isocalendar().year

In [251]:
# Evaluating unique values:
for column in df.columns:
    print(df[column].value_counts())

item
1034.0    2
1015.0    2
1.0       1
708.0     1
707.0     1
         ..
359.0     1
360.0     1
361.0     1
362.0     1
1059.0    1
Name: count, Length: 1058, dtype: int64
mes_registro
enero         379
febrero       363
marzo         258
abril          65
diciembre       4
agosto          1
septiembre      1
noviembre       1
Name: count, dtype: int64
convenio
consorcio    14
Name: count, dtype: int64
gestión_vacante_
gestionar        1066
sin gestionar       5
cancelada           1
Name: count, dtype: int64
población
regular                 994
discapacidad             58
regular/discapacidad      9
discapacidad/regular      6
regular/migrante          5
Name: count, dtype: int64
no.
54.0    47
42.0    41
55.0    30
23.0    29
47.0    27
        ..
25.0     7
69.0     7
68.0     5
71.0     2
72.0     1
Name: count, Length: 72, dtype: int64
código_proceso
311612-51         33
211626-95         33
314594-62566      24
1626050035-58     22
1625936461-29     13
                  ..


In [252]:
# Converting to integer
df['colocado'] = pd.to_numeric(df['colocado'], errors='coerce')
df['colocado'] = df['colocado'].astype(float).fillna(0).astype(int)

# Working with dates
df['mes_publicacion_vacante'] = df['fecha_de_publicacion_de_la_vacante'].dt.month
df['mes_publicacion_vacante'] = pd.to_numeric(df['mes_publicacion_vacante'], errors='coerce')
df['mes_publicacion_vacante'] = df['mes_publicacion_vacante'].fillna(0).astype(int)

## CALCULO DE POBLACIONES

In [253]:
# # Población con discapacidad
# df['discapacidad'] = ""

# # Filters applied
# filter_discapacidad = (
#     (df['condicion'].str.contains('Cognitiva')) |
#     (df['condicion'].str.contains('Física')) |
#     (df['condicion'].str.contains('Visual')) |
#     (df['condicion'].str.contains('Auditiva')) |
#     (df['condicion'].str.contains('Múltiple')) |
#     (df['condicion'].str.contains('Psicosocial')) |
#     (df['condicion'].str.contains('Sordoceguera')) |
#     (df['condicion'].str.contains('pcd')) |
#     (df['condicion'].str.contains('p.c.d')) |
#     (df['condicion'].str.contains('capacid')) 
# )

# df.loc[filter_discapacidad,'discapacidad'] = 'pcd'

# # Replace empty strings with NaN
# df['discapacidad'] = df['discapacidad'].replace("", np.nan)

In [254]:
# Initialize the column with NaN directly
df['discapacidad'] = pd.Series([np.nan] * len(df), dtype='object')

# Mapping of patterns to labels
discapacidad_patterns = {
    r'ognitiv|telect': 'Cognitiva o Intelectual',
    r'[ií]sic': 'Física',
    r'visual': 'Visual',
    r'auditiva': 'Auditiva',
    r'múltiple': 'Múltiple',
    r'sordoceguera': 'Sordoceguera',
    r'psicosocial': 'Psicosocial',
    # r'capacidad': 'Discapacidad'
}

# Apply patterns
for pattern, label in discapacidad_patterns.items():
    mask = df['condicion'].str.contains(pattern, case=False, na=False)
    df.loc[mask, 'discapacidad'] = label

In [255]:
# Poblacion VCA
df['vca'] = pd.Series(dtype='object') # Adding a new column

filter_vca = (
    df['condicion'].str.contains('vca|v.c.a|ctima', na=False) 
)

df.loc[filter_vca,'vca'] = 'vca'

# Replace empty strings with NaN
df['vca'] = df['vca'].replace("", np.nan)

In [256]:
# df[df['condicion'].str.contains('mch')]

In [257]:
# Población Víctima de Violencia de Género
df['vvg'] = pd.Series(dtype='object')

filter_vvg = (
    df['condicion'].str.contains('vvg|violencia|mch', na=False)
)

df.loc[filter_vvg,'vvg'] = 'vvg'

# Replace empty strings
df['vvg'] = df['vvg'].replace("", np.nan)

In [258]:
# Población migrante y retornada
df['migrante'] = ""

# Filters applied
filter_migrante = (
    (df['condicion'].str.contains('igran')) |
    (df['condicion'].str.contains('etorn')) 
)

df.loc[filter_migrante,'migrante'] = 'Migrante y/o retronado'

# Replace empty strings with NaN
df['migrante'] = df['migrante'].replace("", np.nan)

In [259]:
# Grupos etnicos
df['condicion'] = df['condicion'].fillna('')

df['etnias'] = ""

# Filters applied
filter_afro = (
    (df['condicion'].str.contains('negr')) |
    (df['condicion'].str.contains('afro')) |
    (df['condicion'].str.contains('mulat')) |
    (df['condicion'].str.contains('palen')) 
)

filter_raizal = (
    (df['condicion'].str.contains('raiz'))
)

filter_indig = (
    (df['condicion'].str.contains('indí'))
)

filter_git = (
    (df['condicion'].str.contains('git'))
)

# Column etnic groups
df.loc[(filter_afro | filter_git | filter_indig | filter_raizal),'etnias'] = "grupo_etnico"

# Replace empty strings with NaN
df['etnias'] = df['etnias'].replace("", np.nan)

In [260]:
# Poblacion Reincorporada
df['reincorporados'] = pd.Series(dtype=object)

filter_rei = (
    df['condicion'].str.contains('rein', na=False)
)

df.loc[filter_rei,'reincorporados'] = "reincorporados"

# Cleaning the column
df['reincorporados'] = df['reincorporados'].replace('', np.nan)

## REPORTE DE INDICADORES

In [261]:
# Vacantes registradas
mes_registro = 'marzo'
f_vacante = (df['mes_registro'] == mes_registro)

print(f'El número de empresas que solicitaron vacantes durante el mes {mes_registro} es: {len(df[f_vacante]['nombre_de_la_empresa'].unique())}')
print(f'El número de vacantes atendidas durante el mes {mes_registro} es: {len(df[f_vacante]['código_proceso'].unique())}')
# print(f'El número de puestos de trabajo atendidos durante el mes {mes_registro} es: {df[f_vacante]['vacantes_solicitadas/puestos_de_trabajo_en_plataforma'].sum()}')
print(f'El número de puestos de trabajo atendidos durante el mes {mes_registro} es: {df[f_vacante]['puestos_de_trabajo_en_plataforma'].sum()}')

El número de empresas que solicitaron vacantes durante el mes marzo es: 36
El número de vacantes atendidas durante el mes marzo es: 62
El número de puestos de trabajo atendidos durante el mes marzo es: 114.0


In [262]:
# Hojas de vida remitidas
mes = 3.0
f_remitidos = (df['fecha_envío_hojas_de_vida_a_la_empresa'].dt.month == mes)

print(f'El número de hombres con HV remitidas durante el mes {mes} es: {df[f_remitidos & (df['sexo']=='m')]['sexo'].count()}')
print(f'El número de mujeres con HV remitidas durante el mes {mes} es: {df[f_remitidos & (df['sexo']=='f')]['sexo'].count()}')
print()

print(f'El número de HV remitidas para PCD durante el mes {mes} es: {df[f_remitidos]['discapacidad'].count()}')
print(f'El número de HV remitidas para victimas durante el mes {mes} es: {df[f_remitidos]['vca'].count()}')
print(f'El número de HV remitidas para victimas de violencia de género durante el mes {mes} es: {df[f_remitidos]['vvg'].count()}')
print(f'El número de HV remitidas para migrantes durante el mes {mes} es: {df[f_remitidos]['migrante'].count()}')
print(f'El número de HV remitidas para grupos étnicos durante el mes {mes} es: {df[f_remitidos]['etnias'].count()}')
print(f'El número de HV remitidas para reincorporados durante el mes {mes} es: {df[f_remitidos]['reincorporados'].count()}')
print()

print(f'El número de HV remitidas para adultos mayores durante el mes {mes} es: {df[f_remitidos & (df['edad'] >= 60)]['sexo'].count()}')
print(f'El número de HV remitidas para adultos durante el mes {mes} es: {df[f_remitidos & (df['edad'] >= 29) & (df['edad'] < 60)]['sexo'].count()}')
print(f'El número de HV remitidas para jovenes durante el mes {mes} es: {df[f_remitidos & (df['edad'] <= 28)]['sexo'].count()}')


El número de hombres con HV remitidas durante el mes 3.0 es: 133
El número de mujeres con HV remitidas durante el mes 3.0 es: 103

El número de HV remitidas para PCD durante el mes 3.0 es: 19
El número de HV remitidas para victimas durante el mes 3.0 es: 7
El número de HV remitidas para victimas de violencia de género durante el mes 3.0 es: 7
El número de HV remitidas para migrantes durante el mes 3.0 es: 3
El número de HV remitidas para grupos étnicos durante el mes 3.0 es: 5
El número de HV remitidas para reincorporados durante el mes 3.0 es: 0

El número de HV remitidas para adultos mayores durante el mes 3.0 es: 0
El número de HV remitidas para adultos durante el mes 3.0 es: 145
El número de HV remitidas para jovenes durante el mes 3.0 es: 91


In [263]:
# Colocados por mes
mes_colocado = 'marzo'
f_colocados = (df['colocado'] == 1)&(df['mes_de_colocación_en_plataforma'] == mes_colocado)

print(f'El número de hombres colocados durante el mes {mes_colocado} es: {df[f_colocados & (df['sexo']=='m')]['sexo'].count()}')
print(f'El número de mujeres colocados durante el mes {mes_colocado} es: {df[f_colocados & (df['sexo']=='f')]['sexo'].count()}')
print()

print(f'El número de colocados para PCD durante el mes {mes_colocado} es: {df[f_colocados]['discapacidad'].count()}')
print(f'El número de colocados para victimas durante el mes {mes_colocado} es: {df[f_colocados]['vca'].count()}')
print(f'El número de colocados para para victimas de violencia de género durante el mes {mes} es: {df[f_colocados]['vvg'].count()}')
print(f'El número de colocados para migrantes durante el mes {mes_colocado} es: {df[f_colocados]['migrante'].count()}')
print(f'El número de colocados para grupos étnicos durante el mes {mes_colocado} es: {df[f_colocados]['etnias'].count()}')
print(f'El número de colocados para reincorporados durante el mes {mes_colocado} es: {df[f_remitidos]['reincorporados'].count()}')
print()

print(f'El número de colocados para adultos mayores durante el mes {mes_colocado} es: {df[f_colocados & (df['edad'] >= 60)]['sexo'].count()}')
print(f'El número de colocados para adultos durante el mes {mes_colocado} es: {df[f_colocados & (df['edad'] >= 29) & (df['edad'] < 60)]['sexo'].count()}')
print(f'El número de colocados para jovenes durante el mes {mes_colocado} es: {df[f_colocados & (df['edad'] <= 28)]['sexo'].count()}')

El número de hombres colocados durante el mes marzo es: 17
El número de mujeres colocados durante el mes marzo es: 15

El número de colocados para PCD durante el mes marzo es: 16
El número de colocados para victimas durante el mes marzo es: 0
El número de colocados para para victimas de violencia de género durante el mes 3.0 es: 0
El número de colocados para migrantes durante el mes marzo es: 1
El número de colocados para grupos étnicos durante el mes marzo es: 0
El número de colocados para reincorporados durante el mes marzo es: 0

El número de colocados para adultos mayores durante el mes marzo es: 0
El número de colocados para adultos durante el mes marzo es: 22
El número de colocados para jovenes durante el mes marzo es: 10


In [264]:
# El número de empresas que contrataron población fueron:
f_poblaciones = ~(df['etnias'].isna() & df['migrante'].isna() & df['vca'].isna() & df['discapacidad'].isna())
print(f'El número de empresas que contrataron poblaciones PCD, VCA, migrantes y Etnias durante el mes {mes_registro} es: {len(df[f_poblaciones & f_colocados]['nombre_de_la_empresa'].unique())}')
print()
print(f'El nombre de las empresas que contrataron poblaciones PCD, VCA, migrantes y Etnias durante el mes {mes_registro} son: ')
print(df[f_poblaciones & f_colocados]['nombre_de_la_empresa'].unique())

El número de empresas que contrataron poblaciones PCD, VCA, migrantes y Etnias durante el mes marzo es: 4

El nombre de las empresas que contrataron poblaciones PCD, VCA, migrantes y Etnias durante el mes marzo son: 
['grupo alimentario del atlantico s.a. gralco s.a.'
 'servicios globales s.a.s.' 'corporacion universitaria americana'
 'fundacion urbe futuro']


# INFORMACIÓN JOSE BRIEVA

In [275]:
# REMITIDOS
mes = 3.0
f_remitidos = (df['fecha_envío_hojas_de_vida_a_la_empresa'].dt.month == mes)

print(f'El número de HV remitidas para PCD durante el mes {mes} es: {df[f_remitidos]['discapacidad'].count()}')
df[f_remitidos].pivot_table(index=['rango_de_edad','sexo'], columns='discapacidad', aggfunc='size', fill_value=0)

El número de HV remitidas para PCD durante el mes 3.0 es: 19


discapacidad        Auditiva  Física
rango_de_edad sexo                  
18-28         f            4       0
              m            0       2
29-39         f            2       1
              m            5       0
40-49         f            3       0
              m            1       0
50-59         f            1       0

In [276]:
# COLOCADOS
mes_colocado = 'marzo'
f_colocados = (df['colocado'] == 1)&(df['mes_de_colocación_en_plataforma'] == mes_colocado)

print(f'El número de colocados para PCD durante el mes {mes_colocado} es: {df[f_colocados]['discapacidad'].count()}')
df[f_colocados].pivot_table(index=['rango_de_edad','sexo'], columns='discapacidad', aggfunc='size', fill_value=0)

El número de colocados para PCD durante el mes marzo es: 16


discapacidad        Auditiva  Física
rango_de_edad sexo                  
18-28         f            3       0
              m            0       1
29-39         f            1       0
              m            6       1
40-49         f            2       0
              m            1       0
50-59         f            1       0

In [277]:
# EMPRESAS QUE DIERON VACANTES
print(f'Los puestos puestos de trabajo gestionados para población de PCD durante el mes {mes} fueron: ')
df[(df['mes_registro']==mes_colocado) & (~df['discapacidad'].isna())].groupby('nombre_de_la_empresa')['puestos_de_trabajo_en_plataforma'].sum()


Los puestos puestos de trabajo gestionados para población de PCD durante el mes 3.0 fueron: 


nombre_de_la_empresa
corporacion universitaria americana    6.0
dolmen s.a.e.s.p.                      2.0
fundacion urbe futuro                  1.0
támara imágenes diagnosticas s.a.s.    1.0
Name: puestos_de_trabajo_en_plataforma, dtype: float64

In [278]:
print(f'Los colocados en empresas que ofrecieron vacantes para población de PCD durante el mes {mes_colocado} fueron: ')
df[f_colocados & (~df['discapacidad'].isna())]['nombre_de_la_empresa'].value_counts()

Los colocados en empresas que ofrecieron vacantes para población de PCD durante el mes marzo fueron: 


nombre_de_la_empresa
corporacion universitaria americana                 12
servicios globales s.a.s.                            2
grupo alimentario del atlantico s.a. gralco s.a.     1
fundacion urbe futuro                                1
Name: count, dtype: int64

In [279]:
# AREAS 
print(f'Las contratación de PcD durante el mes {mes} se dió como:')
df[f_colocados & (~df['discapacidad'].isna())]['nombre_de_la_vacante'].value_counts()

Las contratación de PcD durante el mes 3.0 se dió como:


nombre_de_la_vacante
docente de apoyo -modelo linguistico    12
asistente administrativo                 2
auxiliar hseq th                         1
operario de aseo (pcd)                   1
Name: count, dtype: int64